In [25]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPool2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils

import datetime

In [3]:
# Load data
ds = pd.read_csv('../data/train.csv')
data = ds.values[:12000]

In [11]:
# data for the first 5 digits (0-4)

mask = data[:, 0] < 5
print mask.sum()

# create 2 sets for (0-4) and (5-9)
data_01 = []
data_02 = []

for ix in xrange(data.shape[0]):
    if mask[ix] == True:
        data_01.append(data[ix])
    else:
        data_02.append(data[ix])

data_01 = np.asarray(data_01)
data_02 = np.asarray(data_02)

print data_01.shape, data_02.shape

6134
(6134, 785) (5866, 785)


In [14]:
split = int(0.85 * data_01.shape[0])

X_tr = data_01[:split, 1:]/255.0
X_te = data_01[split:, 1:]/255.0

X_tr = X_tr.reshape((-1, 28, 28, 1))
X_te = X_te.reshape((-1, 28, 28, 1))

y_tr = np_utils.to_categorical(data_01[:split, 0], num_classes=5)
y_te = np_utils.to_categorical(data_01[split:, 0], num_classes=5)

print X_tr.shape, X_te.shape
print y_tr.shape, y_te.shape

(5213, 28, 28, 1) (921, 28, 28, 1)
(5213, 5) (921, 5)


In [24]:
model = Sequential()

model.add(Convolution2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Convolution2D(32, (3, 3), activation='relu'))

model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(16, (3, 3), activation='relu'))
model.add(Convolution2D(8, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
__________

In [29]:
# model.layers[:-4]

start = datetime.datetime.now()
# process
print datetime.datetime.now() - start

0:00:00.000054


In [30]:
start = datetime.datetime.now()
hist = model.fit(X_tr, y_tr,
                epochs=20,
                shuffle=True,
                batch_size=128,
                validation_data=(X_te, y_te))

print datetime.datetime.now() - start

Train on 5213 samples, validate on 921 samples
Epoch 1/20
5213/5213 [==============================] - 15s - loss: 0.5923 - acc: 0.8018 - val_loss: 0.1281 - val_acc: 0.9598

In [36]:
trans_model = Sequential(model.layers[:-4])

for ix in trans_model.layers:
    ix.trainable = False

trans_model.add(Dense(5))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 2565      
__________

In [38]:
split = int(0.85 * data_02.shape[0])

X_tr = data_02[:split, 1:]/255.0
X_te = data_02[split:, 1:]/255.0

X_tr = X_tr.reshape((-1, 28, 28, 1))
X_te = X_te.reshape((-1, 28, 28, 1))

y_tr = np_utils.to_categorical(data_02[:split, 0]-5, num_classes=5)
y_te = np_utils.to_categorical(data_02[split:, 0]-5, num_classes=5)

print X_tr.shape, X_te.shape
print y_tr.shape, y_te.shape

(4986, 28, 28, 1) (880, 28, 28, 1)
(4986, 5) (880, 5)


In [39]:
start = datetime.datetime.now()
hist = trans_model.fit(X_tr, y_tr,
                epochs=10,
                shuffle=True,
                batch_size=128,
                validation_data=(X_te, y_te))

print datetime.datetime.now() - start

Train on 4986 samples, validate on 880 samples
Epoch 1/10
4986/4986 [==============================] - 0s - loss: 1.0865 - acc: 0.6452 - val_loss: 0.3550 - val_acc: 0.8739
Epoch 2/10
4986/4986 [==============================] - 0s - loss: 0.2616 - acc: 0.9160 - val_loss: 0.2020 - val_acc: 0.9341